File: data-samples/tarrant-tx.csv  
Source: https://www.tad.org/data-reports

## Get the data from the website into a data-frame
1. The only clean-up we're doing here is making sure the string data is trimmed
2. The part is idempotent - after running it at least once, delete /tmp/spark/PropertyData.txt to have it use fresh data from the download source

In [10]:
import requests, zipfile, io, os
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, trim, col

spark = SparkSession.builder.master("local[1]").appName("PropertyData").getOrCreate()
zip_url = "https://www.tad.org/content/data-download/PropertyData(Delimited).ZIP"
source_file = "/tmp/spark/PropertyData.txt"

if os.path.isfile(source_file):
    print("Using existing property file")
else:
    download_url = "/tmp/spark/"
    r = requests.get(zip_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(download_url)

df = spark.read.csv(source_file, sep="|", header=True, inferSchema=True)
df = df.withColumn('id', monotonically_increasing_id())
df = df[['id'] + df.columns[:-1]]   # move id column to front
for name, dtype in df.dtypes:       # trim all string columns
    if dtype == "string":
        df = df.withColumn(name, trim(col(name)))

23/06/12 23:00:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Using existing property file


## Start the training.
1. Specify numeric values for learning.
2. Specify categorical values for learning.

In [11]:
train, test = df.randomSplit([0.7, 0.3])
numerical_features = ['Appraisal_Year', 'County', 'City', 'School', 'Num_Special_Dist', 'Spec1', 'Spec2',
                      'Spec3', 'Spec4', 'Land_Value', 'Improvement_Value', 'Total_Value', 'Garage_Capacity',
                      'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built', 'Living_Area', 'Land_Acres', 'Land_SqFt',
                      'Ag_Acres', 'Ag_Value', 'Structure_Count',]

from pyspark.ml.features import Imputer
imputer = Imputer(inputCols=numerical_features, outputCols=numerical_features)
imputer = imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)

ModuleNotFoundError: No module named 'pyspark.ml.features'

In [16]:
spark.stop()